# Ungraded Lab - Building a Doubly Linked List Class with an LLM

Welcome to the first ungraded lab of this course! In this lab you'll be working alongside an LLM to update a Linked List class to make it doubly linked. This is a good opportunity to practice your LLM prompting skills and prepare yourself for the programming assignment at the end of this course.

# Outline
- [ 1 - Introduction](#1)
  - [ 1.1 Importing necessary libraries](#1.1)
- [ 2 - The `Node` and `LinkedList` Classes to Update](#2)
- [ 3 - Test Your Classes](#3)
- [ 4 - Go Further with Your LLM Prompting Skills](#4)

<a name="1"></a>
## 1 - Introduction

**Your Task:** Below you'll find the `Node` and `LinkedList` class you saw in the lectures. Your job is to work alongside an LLM to update this class to be a doubly linked list, meaning each node has connections to both its previous and next node. Once you've done that, work with the LLM to further refine the class to account for other concerns common in software engineering like security concerns or scalability. 

**LLM Access:** You can access OpenAI's GPT-3.5 model [here](https://www.coursera.org/learn/introduction-to-generative-ai-for-software-development/ungradedLab/Vuqvf/gpt-3-5-environment), but feel free to use the LLM you want!

**Practice Prompting:** Focus on trying out the prompting skills covered in the lectures:

* **Be Specific:** In your prompts provide detail about what you're trying to accomplish and the context in which you're working. For example, it'd be totally appropriate to provide the LLM the class as it's already written and describe the new functionality you're trying to add.
* **Provide Feedback:** Iteratively prompt the LLM and provide feedback on the output you receive to get closer to your expected results. In this case, you could try the code you develop alongside the LLM and report back on bugs, unexpected behavior, or stylistic decisions you want improved.
* **Assign a Role:** Assign a role to tailor the output you receive from the LLM. At first you might just want to assign the role of "an experienced Python developer" but later on try out more specific or expert roles to focus on areas like security or scalability. 

**Testing Your Class:** At the bottom of this notebook you'll find different test cases that will help determine if your class works as expected. This lab is ungraded, however, so you don't need to pass all the test cases to move on. Focus instead on exploring what coding alongside an LLM is like, trying the prompting skills, and building your own intuitive sense of how LLMs will best fit into your software development workflow.

<a name="1.1"></a>
### 1.1 Importing necessary libraries

In [1]:
import threading # Used to make the class thread-safe

<a name="2"></a>
## 2 - The `Node` and `LinkedList` Classes to Update
Below are the classes you saw in the lectures and that you will be editing. Recall that a linked list is made up of individual nodes that have connections between one another. This class initially is a singly linked list, meaning each node only knows the location of the node that comes after it in the linked list. In a doubly linked list the nodes also know the location of the node that comes before it. 

**Update both these classes to make the linked list doubly-linked.**

In [2]:
class Node:
    def __init__(self, data):
        """
        Initialize a new node with the given data.
        
        :param data: The data to store in the node.
        """
        self.data = data
        self.next = None
        self.prev = None

class DoublyLinkedList:
    def __init__(self, max_size=None):
        """
        Initialize a new doubly linked list.
        
        :param max_size: The maximum number of elements the list can hold (optional).
        """
        self.head = None
        self.tail = None
        self.size = 0
        self.max_size = max_size
        self.lock = threading.Lock()  # For thread safety

    def add(self, data, position=None):
        """
        Add a new element to the list at the specified position.
        
        :param data: The data to add to the list.
        :param position: The position to add the new element (optional). If not specified, adds to the end.
        """
        with self.lock:  # Ensure thread safety
            if self.max_size is not None and self.size >= self.max_size:
                print("Error: Linked list is full")
                return
            new_node = Node(data)
            if position is None or position >= self.size:
                # Add to the end
                if not self.head:
                    self.head = self.tail = new_node
                else:
                    self.tail.next = new_node
                    new_node.prev = self.tail
                    self.tail = new_node
            else:
                # Add to a specific position
                if position == 0:
                    new_node.next = self.head
                    self.head.prev = new_node
                    self.head = new_node
                else:
                    current = self.head
                    for _ in range(position - 1):
                        current = current.next
                    new_node.next = current.next
                    new_node.prev = current
                    if current.next:
                        current.next.prev = new_node
                    current.next = new_node
            self.size += 1

    def print_element(self, position):
        """
        Print the element at the specified position.
        
        :param position: The position of the element to print.
        """
        with self.lock:
            if position >= self.size or position < 0:
                print("Position out of bounds")
                return
            current = self.head
            for _ in range(position):
                current = current.next
            print(current.data)

    def update(self, position, data):
        """
        Update the element at the specified position with new data.
        
        :param position: The position of the element to update.
        :param data: The new data to update the element with.
        """
        with self.lock:
            if position >= self.size or position < 0:
                print("Position out of bounds")
                return
            current = self.head
            for _ in range(position):
                current = current.next
            current.data = data

    def delete(self, position):
        """
        Delete the element at the specified position.
        
        :param position: The position of the element to delete.
        """
        with self.lock:
            if position >= self.size or position < 0:
                print("Position out of bounds")
                return
            if position == 0:
                self.head = self.head.next
                if self.head:
                    self.head.prev = None
                else:
                    self.tail = None
            else:
                current = self.head
                for _ in range(position - 1):
                    current = current.next
                current.next = current.next.next
                if current.next:
                    current.next.prev = current
                else:
                    self.tail = current
            self.size -= 1

    def print_statistics(self):
        """
        Print statistics about the linked list, including the number of active items and the total memory occupied.
        """
        with self.lock:
            print(f"Number of active items: {self.size}")
            print(f"Total memory occupied: {self.size * self.head.__sizeof__()} bytes")

    def __str__(self):
        """
        Return a string representation of the linked list.
        
        :return: A string representation of the linked list.
        """
        with self.lock:
            elements = []
            current = self.head
            while current:
                elements.append(current.data)
                current = current.next
            return " <-> ".join(map(str, elements))

    def print_list(self):
        """
        Print the elements of the linked list.
        """
        with self.lock:
            current = self.head
            while current:
                print(current.data, end=" ")
                current = current.next
            print()

    def print_list_reverse(self):
        """
        Print the elements of the linked list in reverse order.
        """
        with self.lock:
            current = self.tail
            while current:
                print(current.data, end=" ")
                current = current.prev
            print()

<a name="3"></a>
## 3 - Test Your Classes
Below are three tests that should help you validate that your updated class is working as intended.

In [4]:
# Test 1 - Append Multiple Data Types
# As initially designed not all data types can be added to the linked list.
# Update the code to allow for additional data types.

linked_list = DoublyLinkedList()
linked_list.add("A")
linked_list.add(1)
linked_list.add(0.1)
linked_list.print_list()

# Expected Output:
# A 1 0.1

A 1 0.1 


In [5]:
# Test 2 - Print the Linked List in Reverse
# Write the print_list_reverse method. Once your list is doubly linked
# this should be a much easier method to write

linked_list = DoublyLinkedList()
linked_list.add("A")
linked_list.add("B")
linked_list.add(10)
linked_list.add(20)
linked_list.print_list_reverse()

# Expected Output:
# 20 10 B A

20 10 B A 


In [7]:
%%timeit
# Test 3 - Append 10,000 items rapidly
# As initially written this is a very slow process. Your updated class
# should be able to find the tail of your linked list (the last node)
# very quickly, significantly speeding up this process.
# Runtimes will vary substantially but as initially written the append method
# will take well more than a second. A refactored doubly linked list class
# should take significantly less than a second.

linked_list = DoublyLinkedList()
for i in range(10000):
    linked_list.add("A")

7.06 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


<a name="4"></a>
## 4 - Go Further with Your LLM Prompting Skills

The three tests above are simple checks that your class is doubly linked, but it's by no means comprehensive of every concern you'd have about the design of this class. Take some time to experiment with either additional functionality you'd want to add, or prompt the LLM to suggest additions based on new roles, like one of a security or scalability expert. Remember, the most important part of this activity is building your skills working with an LLM, so come up with interesting ways to test what these tools are able to help you accomplish.